In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

from setup_imports import *  # noqa: F401,F403
from langcodes import Language
from src.shop import ShopifyProductListing, ShopifyProduct
from src.utils import render_html_content, load_template

In [4]:
COLLECTION = "SURVIVAL"
DECK = "Pack01"
TARGET_LANGUAGE = Language.get("de-DE")
print(TARGET_LANGUAGE.display_name())
SOURCE_LANGUAGE = Language.get("en-GB")
print(SOURCE_LANGUAGE.display_name())

German (Germany)
English (United Kingdom)


In [16]:
# Manual Step 3: Generate Shopify CSV
sp = ShopifyProduct.create(
    collection=COLLECTION,
    pack_type="Specialty",
    template="survival_pack_template.html",
    target_language=TARGET_LANGUAGE,
    source_language=SOURCE_LANGUAGE)

In [17]:
sp.image_filenames

['en-gb_de-de_survival_specialty.png', 'flashcard_anatomy.png']

In [18]:
pl = ShopifyProductListing(products=[sp])

In [19]:
pl.to_csv("../outputs/shopify/shopify_product_listing.csv")

'../outputs/shopify/shopify_product_listing.csv'

In [9]:
sp.images

['https://cdn.shopify.com/s/files/1/0925/9630/6250/files/en-gb_de-de_survival_specialty.png',
 'https://cdn.shopify.com/s/files/1/0925/9630/6250/files/flashcard_anatomy.png']

In [8]:
sp.body_html

: 

In [6]:
sp.save_html(filename="../outputs/shop_page.html")

: 

In [ ]:
print(f"Current language: {config.TARGET_LANGUAGE_NAME}")
print(f"Current language code: {config.TARGET_LANGUAGE_CODE}")

In [ ]:
config.TARGET_LANGUAGE_NAME